In [21]:
# this notebook was used in the initial development of the recommender system, and for testing code before full integration
# it does not contain all of the functionality present in the main application.
# It was included in the submission as it provides a clearer step by step implementation
# of the basic recommender system that was implemented into the application.

from sklearn.preprocessing import MinMaxScaler
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
from sklearn.feature_extraction.text import TfidfVectorizer

scope='user-library-read playlist-modify-public playlist-modify-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="6584944ee0d9495480193d9997a9efb7",
                                               client_secret="ac4205f012b64179acd46c0fbdb33f36",
                                               redirect_uri="http://127.0.0.1:5000/redirect",
                                               scope=scope))

songs = []
pl = sp.current_user_playlists()['items'];
id = pl[2]['id']
playlist = sp.playlist_tracks(id)

tracks_data = []
genres_list = []

for track in playlist['items']:
    
    track_data = {
            'danceability': None,
            'energy': None,
            'loudness': None,
            'speechiness': None,
            'acousticness': None,
            'instrumentalness': None,
            'liveness': None,
            'valence': None,
            'tempo': None,
            'genres': None,
            'track_pop': None,
            'id': track['track']['id']
        }
    
    def get_genres(track_id):
        # Get track information
        track_info = sp.track(track_id)
            
        # Get the list of genres for the first artist of the track
        if 'artists' in track_info and track_info['artists']:
            artist_id = track_info['artists'][0]['id']
            artist_info = sp.artist(artist_id)
                
            if 'genres' in artist_info:
                return artist_info['genres']
            
        return None
    
    audio_features = sp.audio_features([track['track']['uri']])[0]
    if audio_features:
        track_data['danceability'] = audio_features['danceability']
        track_data['energy'] = audio_features['energy']
        track_data['loudness'] = audio_features['loudness']
        track_data['speechiness'] = audio_features['speechiness']
        track_data['acousticness'] = audio_features['acousticness']
        track_data['instrumentalness'] = audio_features['instrumentalness']
        track_data['liveness'] = audio_features['liveness']
        track_data['valence'] = audio_features['valence']
        track_data['tempo'] = audio_features['tempo']
        track_id = track['track']['id']
        genres = get_genres(track_id)
        for genre in genres:
            genres_list.append(genre)
        track_info = sp.track(track_id)
        popularity = track_info['popularity']
        track_data['track_pop'] = popularity
        
        tracks_data.append(track_data)

playlist_features = pd.DataFrame(tracks_data)
playlist_features.drop_duplicates('id')

columns_to_scale = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'track_pop']
data_to_scale = playlist_features[columns_to_scale]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)

# Replace the original values with the scaled values in the DataFrame
playlist_features[columns_to_scale] = scaled_data

genres_list = list(set(genres_list))
print(genres_list)
playlist_features.head()


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=0UtnpKaReKUg2GquaSxCyD:
 Max Retries, reason: too many 429 error responses

In [ ]:
import os
import pymysql
import csv

# Update these values with your MySQL RDS connection details
host = 'fyp-db.cpc2i6c84e9b.eu-west-1.rds.amazonaws.com'  # Replace with your RDS endpoint
port = 3306
database = 'fypdatabase'
username = 'admin'
password = 'fypdatabase'

current_dir = os.getcwd()
relative_path = 'data/tracksgenres.csv'
csv_file_path = os.path.join(current_dir, relative_path)

# Establish a connection
conn = pymysql.connect(host=host, port=port, user=username, password=password, database=database)


In [ ]:
cursor = conn.cursor()
cursor.execute("SELECT danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, track_pop, artists, genres, id FROM Music WHERE genres != '[]'")
rows = cursor.fetchall()
cursor.close()
all_songs_df = pd.DataFrame(rows, columns=['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'track_pop', 'artists', 'genres','id'])


In [ ]:
all_songs_df.head(20)

In [ ]:
import ast
import pandas as pd
import os

from sklearn.preprocessing import MinMaxScaler

current_dir = os.getcwd()
relative_path = '../data/tracksgenres.csv'
file_path = os.path.join(current_dir, relative_path)

# Read the CSV file using pandas read_csv
#all_songs_df = pd.read_csv(file_path)

#doest work anymore
#all_songs_df = pd.read_csv(r'C:\Users\alext\OneDrive\Desktop\ml_data\tracksgenres.csv')

#rearrange columns to match the user dataframe
all_songs_df_order = ['artists','genres', 'id', 'track_pop','danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
all_songs_features_order = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo', 'track_pop', 'id']
all_songs_df = all_songs_df[all_songs_df_order]
all_songs_features = all_songs_df[all_songs_features_order]

columns_to_scale = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'track_pop']
data_to_scale = all_songs_features[columns_to_scale]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)
all_songs_features[columns_to_scale] = scaled_data


all_songs_features.head()


In [ ]:
"""
non_empty_genres_df = all_songs_features[all_songs_features['genres'].apply(len) > 0]

# Specify the maximum number of features
max_features = 500  # You can adjust this number based on your memory constraints

# Create the TfidfVectorizer with max_features
tfidf = TfidfVectorizer(max_features=max_features)

# Apply TF-IDF vectorization to non-empty genres
tfidf_matrix = tfidf.fit_transform(non_empty_genres_df['genres'].apply(lambda x: " ".join(x)))

# Create DataFrame from the TF-IDF matrix
genre_df = pd.DataFrame(tfidf_matrix.toarray(), columns=['genre' + "|" + i for i in tfidf.get_feature_names_out()])

# Reset the index of the DataFrame
genre_df.reset_index(drop=True, inplace=True)
"""


In [ ]:
def generate_playlist_feature(complete_feature_set, playlist_df):

    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_playlist_output = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_output.sum(axis = 0)

In [20]:
playlistfeatures = generate_playlist_feature(playlist_features, all_songs_df)

i = 0
while i <= len(playlist_features):
    playlistfeatures[i] = playlistfeatures[i]/len(playlist_features)
    i+=1

playlistfeatures.pop('genres')
# "single song" matrix
print(playlistfeatures)

NameError: name 'playlist_features' is not defined

In [ ]:
#all_songs_df = all_songs_df[all_songs_df['genres'] != "[]"] 

all_songs_df['genres'] = all_songs_df['genres'].apply(ast.literal_eval)

#all_songs_df = all_songs_df[all_songs_df['genres'].apply(lambda x: any(genre in genres_list for genre in x))]

# Display the filtered DataFrame
print(all_songs_df)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def generate_playlist_recos(df, features, nonplaylist_features, genres):

    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df = non_playlist_df[non_playlist_df['genres'].apply(lambda x: any(genre in genres for genre in x))]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [ ]:
print(playlistfeatures)

In [ ]:
all_songs_features.head()

In [ ]:
recos = generate_playlist_recos(all_songs_df, playlistfeatures, all_songs_features, genres_list)

recos.head(20)

